<a href="https://colab.research.google.com/github/spaziochirale/CorsoPythonML/blob/master/TF_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Esempio di Rete Neurale con Tensor Flow: basic classification

In questa lezione costruiremo e addestreremo una rete neurale per la classificazione di immagini di indumenti come, sneaker, magliette o camice.
L'esempio è basato su uno dei tutorial standard proposti sul sito Tensor Flow e utilizza le API di alto livello del package [tf.keras](https://www.tensorflow.org/guide/keras) della libreria TensorFlow.

Keras, come il resto del TensorFlow, è una API di livello industriale che richiede un certo tempo per il suo apprendimento, pertanto non saranno sempre chiari tutti i dettagli, e, anche se cercheremo di fornire le spiegazioni man mano che l'esempio procede, il lettore è invitato a mantenere una visione generale del problema senza preoccuparsi troppo dei dettagli.

La porzione di codice che segue mostra l'importazione delle librerie necessarie.

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

## Importiamo il dataset Fashion MNIST che sarà utilizzato in questa lezione

Utilizzeremo il dataset [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) che contiene 70.000 immagini monocromatiche (in scala di grigio) relative a 10 categorie di capi d'abbigliamento.

Ciascun'immagine è una bitmap di 28 X 28 pixel che raffigura in bassa risoluzione uno specifico prodotto. 

La figura seguente mostra alcune bitmap del dataset.

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <b>Figura 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Esempi Fashion-MNIST</a> (by Zalando, MIT License).<br/>&nbsp;
  </td></tr>
</table>


Il dataset Fashion MNIST è stato creato da *Zalando Research* come dataset alternativo al celebre, ma oramai abusato, dataset [MNIST](http://yann.lecun.com/exdb/mnist/) divenuto una sorta di *Hello World* nell'ambito dei programmi di visione artificiale. Il datast MNIST classico contiene immagini analoghe relative alle cifre da 0 a 9 scritte con calligrafia manuale.

L'impiego del Fashion MNIST rappresenta un'alternativa molto più interessante e complessa da risolvere. Entrambi i dataset sono sufficientemente piccoli da poter gestire la fase di training su un PC in pochi minuti e rappresentano un ottimo punto di partenza in ambito didattico.

Utilizzeremo 60.000 immagini per addestrare la rete e 10.000 immagini per valutare il livello di generalizzazione raggiunto dalla rete.
Il dataset è incluso nella distribuzione della libreria TensorFlow ed è quindi sufficiente importare e caricare i dati, senza doversi preoccupare ulteriormente dell'acquisizione della fonte dei dati.


In [0]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Il metodo *load_data()* dell'oggetto *fashion_mnist* restituisce quattro oggetti, suddivisi in due tuple, di tipo *array NumPy*. 

Un *array NumPy* è una struttura dati definita nell'ambito della libreria *numpy* molto utilizzata nella *data science*. Per il momento possiamo immaginarli come semplici vettori multidimensionali di elementi senza preoccuparci ulteriormente delle effettive proprietà dell'oggetto.


* Gli array `train_images` e `train_labels` costituiscono il *training set*—che sarà utilizzato per addestrare la rete.
* Una volta addestrato, il modello sarà testato utilizzando il *test set* costituito dagli array `test_images` e `test_labels`.

Le immagini sono array NumPy di dimensione  28x28, i cui valori dei pixel variano da 0 a 255.  

Le etichette *labels* sono array di numeri interi i cui valori variano da 0 a 9. 

Questi valori corrispondono alla *categoria* del capo di abbigliamento raffigurato nell'immagine, secondo la seguente tabella:


<table>
  <tr>
    <th>Label</th>
    <th>Categoria</th> 
  </tr>
  <tr>
    <td>0</td>
    <td>T-shirt/top</td> 
  </tr>
  <tr>
    <td>1</td>
    <td>Pantalone</td> 
  </tr>
    <tr>
    <td>2</td>
    <td>Pullover</td> 
  </tr>
    <tr>
    <td>3</td>
    <td>Abito</td> 
  </tr>
    <tr>
    <td>4</td>
    <td>Cappotto</td> 
  </tr>
    <tr>
    <td>5</td>
    <td>Sandalo</td> 
  </tr>
    <tr>
    <td>6</td>
    <td>Camicia</td> 
  </tr>
    <tr>
    <td>7</td>
    <td>Sneaker</td> 
  </tr>
    <tr>
    <td>8</td>
    <td>Borsa</td> 
  </tr>
    <tr>
    <td>9</td>
    <td>Stivaletto</td> 
  </tr>
</table>

Ciascuna immagine è messa in corrispondenza ad una singola etichetta (label).

Poiché il dataset non include i nome delle categorie, ma solo l'indice numerico da 0 a 9, decidiamo di memorizzare tali nomi in una lista per poi poterli utilizzare per etichettare in modo più comprensibile i grafici che andremo a stampare durante l'esercizio:


In [0]:
class_names = ['T-shirt/top', 'Pantalone', 'Pullover', 'Abito', 'Cappotto', 
               'Sandalo', 'Camicia', 'Sneaker', 'Borsa', 'Stivaletto']

## Exploriamo i nostri dati

Lanciamo ora alcune istruzioni di utilità per esplorare il formato del nostro dataset prima di procedere con la costruzione del modello, cioè della rete, e con la fase di addestramento.

Il comando seguente mostra che il dataset è costituito da 60.000 immagini, con ciascuna immagine costituita da 28X28 pixel. Il metodo *shape* restituisce una tupla che contiene le dimensioni del *tensore*.


In [0]:
train_images.shape

Analogamente utilizziamo questa volta l'operatore *len()* per mostrare che il training set contiene 60.000 label:

In [0]:
len(train_labels)

Ciascuna label è un numero intero tra 0 e 9:

In [0]:
train_labels

10.000 images sono contenute nel test set e, come in precedenza, ciascuna immagine è composta da 28 x 28 pixel:

In [0]:
test_images.shape

Il test set contiene 10.000 etichette:

In [0]:
len(test_labels)

## Preprocessamento dei dati

Per una maggiore efficienza computazionale è sempre raccomandabile di procedere ad un preprocessamento dei dati, normalizzandoli secondo opportuni criteri.

Nel nostro caso se osserviamo la prima immagine del training set vediamo che i valori dei pixel cadono all'interno dell'intervallo 0-255.

Possiamo visualizzare graficamente questo fatto utilizzando opportune funzioni del package *PyPlot* che abbiamo già incontrato nella precedente lezione:


In [0]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

Una normalizzazione molto comune consiste nello scalare l'intervallo dei valori di input nell'ambito dell'intervallo 0-1.

Per far questo è sufficiente dividere ciascun valore per 255.

È importante che entrambi i set di training e test vengano preprocessati nello stesso identico modo.

Grazie alla potenza espressiva del Python l'operazione è pressoché immediata:


In [0]:
train_images = train_images / 255.0

test_images = test_images / 255.0

A questo punto possiamo provare a disegnare le prime 25 immagini del training set riportando il nome della categoria del prodotto sotto ciascuna immagine.

Verifichiamo quindi che i dati siano nel formato corretto in modo da essere pronti a costruire la rete e procedere con l'addestramento.

Anche in questo caso utilizziamo le funzioni messe a disposizione del modulo *PyPlot*. Per approfondimenti il lettore è invitato a consultare la reference del package *PyPlot*


In [0]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## Costruiamo il modello

Nella figura è indicato il modello di rete che abbiamo deciso di utilizzare per risolvere il nostro problema di classificazione.

![Rete Neurale](https://www.chirale.it/wp-content/uploads/2020/03/ReteTFBasic.jpg)

L'input è costituito dai 784 (28 X 28) valori corrispondenti ai singoli pixel dell'immagine che si vuole classificare.

La prima operazione logica consiste quindi nel trasformare, o meglio rimappare, la bitmap che fino ad ora abbiamo visualizzato come una matrice bidimensionale, in un vettore di 784 neuroni.

L'operazione è indicata in figura come *flatten* e, come vedremo, la libreria TensorFlow fornisce la possibilità di effettuare l'operazione in modo automatico, senza doversi preoccupare di implementare il ciclo di lettura della matrice, riga per riga.

Dopo lo strato di **input** abbiamo deciso di collocare uno strato **hidden** di 128 neuroni. Abbiamo scelto di implementare una tipica rete *Deep Neural Network* e pertanto tutti i neuroni di input dovranno essere connessi a tutti i neuroni dello strato successivo. Tale proprietà si indica definendo il secondo strato come ***dense layer***.
Per i neuroni di questo strato, utilizzeremo come funzione di attivazione la funzione *rectifier*, che viene solitamente indicata come ***ReLU***, Rectifier Linear Unit.

Dopo lo strato hidden collochiamo lo strato di **output** costituito da 10 neuroni, anche questo di tipo ***dense***. Abbiamo scelto una tipica soluzione per un classificatore a classi multiple. Ciascun neurone di output corrisponde ad una dicersa classe e il suo valore rappresenta la probabilità che la bitmap in ingresso raffiguri un elemento corrispondente a tale classe.

In questo caso si utilizza come funzione di attivazione la funzione **softmax** che garantisce che la somma degli output e quindi delle probabilità sia pari ad 1.

Procediamo quindi con l'implementazione in Python.

Per costruire una rete neurale attraverso la libreria Keras, si procede configurando i singoli strati del modello e successivamente *compilando* il modello. 


### Configurazione dei layer

Il layer è l'elemento base di una rete neurale.

Nei modelli di deep learning i layer sono collegati gli uni agli altri.

Keras offre dei modelli di layer, come ad esempio `tf.keras.layers.Dense`, che permettono di configurare strati corrispondenti a modelli ben noti nella teoria delle reti neurali e che contengono tutti i parametri che verranno valorizzati a seguito della fase di addestramento.



In [0]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

Il primo layer della rete, `tf.keras.layers.Flatten`, trasforma il formato delle immagini da un array bidimensionale (di 28 X 28 pixel), ad un array monodimensionale di 28 * 28 = 784 pixel. 

Il layer di *flattening* non ha pesi che vengono modificati durante il training. Semplicemente cambia la forma dell'input attraverso un trasferimento diretto del valore in input verso l'output.

Dopo lo strato di allineamento, la rete è composta da una sequenza di due layer `tf.keras.layers.Dense`. Si tratta di strati in cui tutti i neuroni sono connessi. Il primo layer `Dense` ha 128 nodi (o neuroni). Il secondo (e ultimo) layer è un layer  *softmax* con 10 nodi che restituisce un array di 10 punteggi di probabilità, la cui somma è pari ad 1. Il valore in output di ciascuno di questi neuroni rappresenta la probabilità che l'immagine in input sia appartenente alla categoria rappresentata da quel nodo.

### Compilazione del modello

Prima che il modello sia pronto per la fase di training è necessario effettuare ancora alcune configurazioni. Nella piattaforma Keras queste configurazioni vengono svolte attraverso l'operazione di *compilazione* svolta invocando il metodo *compile()*.

* *Loss function* —Serve per misurare l'accuratezza del modello durante la fase di addestramento. L'obiettivo è quello di minimizzare questa funzione in modo da guidare il modello verso la corretta impostazioni dei parametri (pesi).
* *Optimizer* —Rappresenta la funzione attraverso la quale i pesi vengono modificati in base al valore dell'input fornito e alla valutazione della funzione *loss*.
* *Metrics* —Viene utilizzata per monitorare la fase di training e la fase di test. Nel nostro esempio utilizzeremo *accuracy*, valutata come la percentuale di immagini correttamente classificata.

Grazie alla libreria TensorFlow non dobbiamo preoccuparci dei dettagli implementativi delle funzioni matematiche che sono alla base del calcolo del *loss* e della strategia di ottimizzazione, ma possiamo limitarci a richiamare metodi noti in letteratura e già implementati nelle API.

In [0]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Addestramento del Modello

La fase di training della rete è composta dai seguenti passi:

1. Alimentare la rete con i dati di training—nel nostro caso, gli array `train_images` e `train_labels`.
2. Attendere che il modello apprenda la corretta associazione tra immagini e *lable*.
3. Sperimentare le prestazioni del modello alimentandolo con l'insieme dei dati di test—nel nostro caso l'array `test_images` . In particolare verificheremo che le immagini siano associate correttamente alle etichette contenute nell'array `test_labels`. 

Il training viene avviato invocando il metodo `model.fit` —il termine "fit" è tipico del machine learning basato su metodi statistici. Il metodo riceve in inpute i due array e il numero di *epoche* da utilizzare per l'addestramento:

In [0]:
model.fit(train_images, train_labels, epochs=10)

Man mano che procede il training, vengono visualizzati sullo schermo i valori della funzione loss e l'accuratezza (metrica *accuracy*).

Questo modello raggiunge un'accuratezza di circa l'88% (0,88) sui dati di training.


## Valutare l'accuratezza

Per valutare la bontà del modello occorre misurare la metrica di accuratezza sul dataset di test, composto da dati che non sono mai stati sottoposti alla rete in fase di training. Solo in questo modo potremo stimare quanto  la rete abbia realmente appreso in modo generalizzato.

A tale scopo invochiamo il metodo `evaluate()` passando come parametri gli array `test_images` e `test_labels`.

In [0]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)

print('Test accuracy:', test_acc)

Come si può vedere, l'accuratezza misurata sui dati di test è inferiore all'accuratezza misurata durante il training.

Questo è un esempio di *overfitting*. La rete in pratica ha ben memorizzato i dati di training ma non ha ancora ben generalizzato la sua capacità di classificare dati che non ha mai visto.

Nel nostro caso le performance sono comunque accettabili. Esistono tecniche per gestire e prevenire questo problema.


## Effettuare previsioni

Una volta che la rete è stata addestrata, questa può essere utilizzata per effettuare previsioni.

È quindi sufficiente invocare il metodo `predict()` fornendo come parametro un array di dati. Il metodo è strutturato per gestire batch di dati, ma ovviamente è possibile passare in input un solo dato, creando un array con un solo campione.

Possiamo provare ad invocarlo utilizzando come esempio l'array `test_images`.


In [0]:
predictions = model.predict(test_images)

Il metodo predict() restituisce un array di previsioni. Proviamo quindi a stampare la prima previsione, quella corrispondente alla prima immagine fornita in input.


In [0]:
predictions[0]

Ciascuna previsione è a sua volta un array di 10 valori. Ciascun valore indica la probabilità che l'immagine di input appartenga alla categoria corrispondente a quell'indice.

Se vogliamo estrarre la lable corrispondente al valore di probabilità più alto, possiamo usare la funzione `argmax()` del package NumPy.


In [0]:
np.argmax(predictions[0])

Il modello è maggiormente confidente che l'immagine sia uno stivaletto, cioè `class_name[9]`. 

Nel nostro caso possiamo verificare quale sia la lable corretta dall'array `test_lable`.


In [0]:
test_labels[0]

Possiamo creare un grafico per visualizzare l'insieme dei 10 canali.

In [0]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  
  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'
  
  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1]) 
  predicted_label = np.argmax(predictions_array)
 
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

Osserviamo l'immagine 0, le previsioni e l'array di previsioni.


In [0]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions,  test_labels)
plt.show()

In [0]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions,  test_labels)
plt.show()

Effettuiamo un grafico con diverse immagini e le relative previsioni.
Le previsioni corrette sono di colore blu quelle errate di colore rosso.
I numeri forniscono le percentuali di confidenza per la lable stimata. Si osservi che anche nel caso in cui la rete stimi una elevata confidenza per una certa *lable*, questa può essere errata.


In [0]:
# Plot the first X test images, their predicted label, and the true label
# Color correct predictions in blue, incorrect predictions in red
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, test_labels)
plt.show()

Proviamo ora ad utilizzare la rete addestrata per effettuare una previsione utilizzando una sola immagine.


In [0]:
# Grab an image from the test dataset
img = test_images[0]

print(img.shape)

Come già detto, i modelli `tf.keras` sono progettati per effettuare previsioni su *batch*, o collezioni, di esempi, forniti in blocco. Pertanto, se vogliamo utilizzare una sola immagine, questa deve essere comunque inserita in una lista.

In [0]:
# Add the image to a batch where it's the only member.
img = (np.expand_dims(img,0))

print(img.shape)

Effettuiamo la previsione per l'immagine:

In [0]:
predictions_single = model.predict(img)

print(predictions_single)

In [0]:
plot_value_array(0, predictions_single, test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)

`model.predict` restituisce una lista di liste, una lista per ogni singola immagine del batch fornito. Estraiamo quindi la previsione per l'unica immagine di cui è costituito il batch:

In [0]:
np.argmax(predictions_single[0])

Ovviamente, come già visto, la previsione è 9.